In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
def select_CHT_edges_table(database_name, ParseDate):
    conn = sqlite3.connect(database_name)
    cur = conn.cursor()
    myresult = cur.execute("SELECT News1,News2 FROM CHTconnection WHERE ParseDate==?", (ParseDate,))
    result = []
    for row in myresult:
        result.append(row)
    conn.close()        
    return result

In [3]:
def select_CHT_nodes_table(database_name, NewsIndex):
    conn = sqlite3.connect(database_name)
    cur = conn.cursor()
    myresult = cur.execute("SELECT NewsIndex,NewsTitle,NewsURL FROM CHTnews WHERE NewsIndex==?", (NewsIndex,))
    result = []
    for row in myresult:
        result.append(row)
    conn.close()    
    return result

In [8]:
DB_PATH = '/Users/gtingyou/Documents/NewsNetwork/Recommendation System/IsorankBased/DailyNews/NewsNetwork_ch.db'
DATE = '2020-04-06'

In [9]:
# 抓每日大網路圖 做network embedding
CHTconnection = select_CHT_edges_table(DB_PATH, DATE)

In [10]:
len(CHTconnection)

12672

In [11]:
def Output_CHTconnection_To_Edgelist_File(CHTconnection):
    df_edgelist = pd.DataFrame(CHTconnection)
    df_edgelist.to_csv('./GraphEmbedding/data/news/news_%s.edgelist' %(DATE),sep=' ',index=None,header=None)

Output_CHTconnection_To_Edgelist_File(CHTconnection)

### node2vec embedding

In [7]:
def Node2Vec_Network_Embedding():
    import numpy as np
    import networkx as nx
    from GraphEmbedding.ge.classify import read_node_label,Classifier
    from GraphEmbedding.ge import Node2Vec
    
    G = nx.read_edgelist('./GraphEmbedding/data/news/news_%s.edgelist' %(DATE),
                        create_using = nx.DiGraph(), nodetype = None, data = [('weight', int)])#read graph

    model = Node2Vec(G, walk_length = 10, num_walks = 80,p = 0.25, q = 4, workers = 1)#init model
    model.train(window_size = 5, iter = 3)# train model
    embeddings = model.get_embeddings()# get embedding vectors

    return embeddings

In [8]:
node2vec_embeddings = Node2Vec_Network_Embedding()

Preprocess transition probs...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s finished


Learning embedding vectors...
Learning embedding vectors done!


In [22]:
emb_dict = node2vec_embeddings

In [9]:
def Output_Embeddings_To_File(embeddings):
    import pandas as pd
    df_embeddings = pd.DataFrame(embeddings).T
    df_embeddings= df_embeddings.sort_index()

    df_embeddings.to_csv('./GraphEmbedding/emb/news_%s.emb' %(DATE),sep=' ',index=True,header=None)

In [10]:
Output_Embeddings_To_File(node2vec_embeddings)

#### 1.獲取使用者瀏覽紀錄

In [11]:
def select_USER_nodes_table(database_name, ParseDate):
    conn = sqlite3.connect(database_name)
    cur = conn.cursor()
    myresult = cur.execute("SELECT NewsIndex,NewsTitle,NewsURL FROM NODES WHERE ParseDate==?", (ParseDate,))
    result = []
    for row in myresult:
        result.append(row)
    conn.close()    
    return result

In [445]:
userlogs = select_USER_nodes_table('/Users/gtingyou/Documents/NewsNetwork/Recommendation System/IsorankBased/DailyUserLogs/user1.db',
                                    '2020-04-03')

#### 2.找出使用者瀏覽紀錄在大網路圖中的NewsIndex

In [458]:
def Get_Userlogs_NewsIndex_In_BigNetwork(userlogs):
    def clean_CHT_url(url):
        if '?chdtv' in url:
            url = url.replace('?chdtv', '')
        if '?ctrack' in url:
            url = url[:url.find('?ctrack')]
        return url
    
    def get_CHT_big_network_index(database_name, NewsURL):
        conn = sqlite3.connect(database_name)
        cur = conn.cursor()
        myresult = cur.execute("SELECT NewsIndex,NewsTitle,NewsURL FROM CHTnews WHERE NewsURL like ?", ('%'+NewsURL+'%',))
        result = []
        for row in myresult:
            result.append(row)
        conn.close()    
        return result
    
    userlogs_NewsIndex = []
    for log in user_logs:
        if log[2]=='https://www.chinatimes.com/':
            continue

        result = get_CHT_big_network_index(DB_PATH, log[2])
        if result!=[]:
            userlogs_NewsIndex.append(result[0])
        else: # 使用者瀏覽紀錄的網址，不在大網路圖中，所以找不到對應的 NewsIndex
            print(log[1],clean_CHT_url(log[2]))
            
    return userlogs_NewsIndex

In [459]:
userlogs_NewsIndex = Get_Userlogs_NewsIndex_In_BigNetwork(userlogs)

print('\nNewsIndex in user browsing history:')
for i in userlogs_NewsIndex: 
    print('-'*50)
    print(select_CHT_nodes_table(DB_PATH, i[0])[0])

疫情惡化 全球確診將破百萬 - 國際大事 https://www.chinatimes.com/newspapers/20200403000278-260119
桃園2同住居家檢疫者 陳屍家中 - 社會 https://www.chinatimes.com/realtimenews/20200403003492-260402
遊客群聚一窩蜂 社交距離破功 - 焦點要聞 https://www.chinatimes.com/newspapers/20200403000298-260114
紐約確診8.4萬人 比陸全國還多 - 焦點要聞 https://www.chinatimes.com/newspapers/20200403000314-260119
 https://www.chinatimes.com/newspapers/2601
 https://www.chinatimes.com/newspapers/2601?page=2
 https://www.chinatimes.com/newspapers/2601?page=2&chdtv
港防疫升級 酒吧今起關閉半個月 - 焦點要聞 https://www.chinatimes.com/newspapers/20200403000337-260108
疫情惡化 美不再禁止陸製KN95口罩進口 - 國際 https://www.chinatimes.com/realtimenews/20200403003589-260408
封國前兆？美國務院發推文：美國人快回來！ - 國際 https://www.chinatimes.com/realtimenews/20200403003659-260408
封國前兆？美國務院發推文：美國人快回來！ - 國際 https://www.chinatimes.com/realtimenews/20200403003659-260408
美Q2經濟慘到爆 衰幅恐達38％ - 財經 https://www.chinatimes.com/realtimenews/20200403003847-260410
美Q2經濟慘到爆 衰幅恐達38％ - 財經 https://www.chinatimes.com/realtimenews/20200403003847-260410

Ne

#### 3.計算推薦新聞文檔相似度

In [16]:
import jieba
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import time

In [17]:
def Open_stopWords_From_File(path):
    stopWords=[]
    # 讀入停用詞檔
    with open('stopWords.txt', 'r', encoding='UTF-8') as file:
        for data in file.readlines():
            data = data.strip()
            stopWords.append(data)
    return stopWords

stopWords = Open_stopWords_From_File(path='./')

In [413]:
def Calculate_News_Document_Similarity(emb_dict):
    '''
    1. get NewsDetail(include NewsIndex,NewsContext,NewsTitle) from database ----> time consuming
    2. construct cosine similarity matix of context and title respectively
    3. compare 0:context and 1:title doc sim score, return the bigger one
    return the final cosine similarity matrix of all the News in Big network
    '''
    ### 1. get NewsDetail
    def select_CHT_nodes_title_context(database_name, NewsIndex):
        conn = sqlite3.connect(database_name)
        cur = conn.cursor()
        myresult = cur.execute("SELECT NewsIndex,NewsContext,NewsTitle FROM CHTnews WHERE NewsIndex==?", (NewsIndex,))
        result = []
        for row in myresult:
            result.append(row)
        conn.close()    
        return result
    
    CS_index_list = sorted(list(emb_dict.keys()))
    News = []
    for index in tqdm( CS_index_list ):
        News.append(list(select_CHT_nodes_title_context(DB_PATH, index)[0]))
    print('End loading NewsTitle and NewsContext from database ---> ', time.strftime("%H:%M:%S", time.localtime()) )
    
    ### construct cs matrix
    def Construct_CS_Matrix(context_or_title, News): # 算 context_or_title 的詞頻矩陣 ---> 1: context , 2: title
        corpus = []
        for i, k in enumerate(News):
            # 結巴中文斷詞
            segments = jieba.cut(k[context_or_title], cut_all=False)
            # 移除停用詞及跳行符號
            remainderWords = list(filter(lambda a: a not in stopWords and a != '\n', segments))
            corpus.append(' '.join(remainderWords))

        vectorizer = CountVectorizer()
        X = vectorizer.fit_transform(corpus)
        cs = cosine_similarity(X)
        return cs    
    
    cs1 = Construct_CS_Matrix(1, News)
    print('End constructing cosine similarity matrix for NewsContext ---> ', time.strftime("%H:%M:%S", time.localtime()) )
    cs2 = Construct_CS_Matrix(2, News)
    print('End constructing cosine similarity matrix for NewsTitle ---> ', time.strftime("%H:%M:%S", time.localtime()) )
    
    ### compare scores
    compared_cs = np.zeros([len(cs1),len(cs1)])
    for i in range(len(cs1)):
        for j in range(len(cs1)):
            if cs1[i][j]>=cs2[i][j]:
                compared_cs[i][j] = cs1[i][j]
            else:
                compared_cs[i][j] = cs2[i][j]
    return CS_index_list, compared_cs

In [414]:
document_similarity_matrix_index, document_similarity_matrix = Calculate_News_Document_Similarity(emb_dict)

100%|██████████| 5229/5229 [01:24<00:00, 62.11it/s]


End loading NewsTitle and NewsContext from database --->  15:36:30
End constructing cosine similarity matrix for NewsContext --->  15:37:13


In [420]:
def Similar_Document_Content_News(NewsID,document_similarity_matrix,document_similarity_matrix_index):
    row_num = document_similarity_matrix_index.index(NewsID)
#     print(row_num)
    doc_similarity_scores = [[document_similarity_matrix_index[i], 
                              document_similarity_matrix[row_num][i]] for i in range(len(document_similarity_matrix)) if i!=row_num] #排除自己
    return sorted(doc_similarity_scores, key=lambda x:x[0], reverse=False)

In [421]:
# doc_similarity_scores = Similar_Document_Content_News('CHT_20200406_2366',
#                                                       document_similarity_matrix,document_similarity_matrix_index)

#### 4.利用node2vec embedding結果，計算topology相似度

In [337]:
def Similar_Topological_News(NewsID, emb_dict):
    import numpy as np
    from sklearn.metrics.pairwise import cosine_similarity
    
    v1 = emb_dict[NewsID] # News1 embedding
    topology_similarity_scores = []
    for i in emb_dict:
        v2 = emb_dict[i] # News2 embedding
        if list(v1)!=list(v2): 
            s = cosine_similarity( np.array(v1).reshape(1,128), np.array(v2).reshape(1,128) )
            topology_similarity_scores.append([i, s[0][0]])
    
    return sorted(topology_similarity_scores, key=lambda x:x[0], reverse=False)

In [338]:
# topology_similarity_scores = Similar_Topological_News(NewsID, emb_dict)

#### 5.根據 *alpha x 文檔相似度+(1-alpha) x topology相似度* 推薦新聞

In [434]:
def Merge_Document_and_Topology_Score(doc_similarity_scores, topology_similarity_scores, standardize):
    alpha = 0.5

    if standardize==True: ### 對 document score和 topology score做標準化
        def Standardization(data):
            mu = np.mean(data, axis=0)
            sigma = np.std(data, axis=0)
            print('mean=%f, sigma=%f' %(mu, sigma))
            return (data - mu) / sigma
        print('Standardize document sim score:')
        standardized_doc_score = Standardization(np.array([k[1] for k in doc_similarity_scores],dtype = 'float'))
        doc_similarity_scores = [[doc_similarity_scores[i][0],
                                  standardized_doc_score[i]] for i in range(len(doc_similarity_scores))]
        print('Standardize topology sim score:')
        standardized_topology_score = Standardization([k[1] for k in topology_similarity_scores])
        topology_similarity_scores = [[topology_similarity_scores[i][0],
                                       standardized_topology_score[i]] for i in range(len(topology_similarity_scores))]

    ### 按照index編號順序排列
    doc_similarity_scores = sorted(doc_similarity_scores, key=lambda x:x[0], reverse=False)
    topology_similarity_scores = sorted(topology_similarity_scores, key=lambda x:x[0], reverse=False)
    ### 依公式 alpha x 文檔相似度+(1-alpha) 計算合併的成績
    merged_similarity_scores = []
    for i in range(len(doc_similarity_scores)):
        if doc_similarity_scores[i][0]==topology_similarity_scores[i][0]: # 檢查index編號是否相同
            merged_similarity_scores.append([doc_similarity_scores[i][0], 
                                             alpha*float(doc_similarity_scores[i][1])+(1-alpha)*topology_similarity_scores[i][1] ])
    
    merged_similarity_score_dict = {merged_similarity_scores[i][0]:[merged_similarity_scores[i][1],
                                                                    doc_similarity_scores[i][1],
                                                                    topology_similarity_scores[i][1]] for i in range(len(merged_similarity_scores) ) }
    
    return merged_similarity_scores, merged_similarity_score_dict

In [435]:
# merged_similarity_scores, merged_similarity_score_dict = Merge_Document_and_Topology_Score(doc_similarity_scores,
#                                                                                           topology_similarity_scores,
#                                                                                           True)

In [440]:
def Recommend_News(NewsID, num_of_recommend_news):
    print('User browsing history News:\n', select_CHT_nodes_table(DB_PATH, NewsID)[0])
    
    doc_similarity_scores = Similar_Document_Content_News(NewsID,
                                                          document_similarity_matrix,
                                                          document_similarity_matrix_index)
    topology_similarity_scores = Similar_Topological_News(NewsID, emb_dict)
    merged_similarity_scores, merged_similarity_score_dict = Merge_Document_and_Topology_Score(doc_similarity_scores,
                                                                                              topology_similarity_scores,
                                                                                              False)
    print('\nRecommed News:')
    for i, k in enumerate(sorted(merged_similarity_scores, key=lambda x:x[1], reverse=True)):
        if i>num_of_recommend_news:
            break
        print('-'*50)
        s = merged_similarity_score_dict[k[0]]
        print('total_sim_score=%s, doc_sim_score=%s, topology_sim_score=%s' %(s[0], s[1], s[2]))
        print(select_CHT_nodes_table(DB_PATH, k[0])[0])

In [454]:
Recommend_News('CHT_20200406_1223', 10)

User browsing history News:
 ('CHT_20200406_1223', '美CDC主任：無症狀感染者多達25％ 疫情或持續2年 - 兩岸', 'https://www.chinatimes.com/realtimenews/20200402004406-260409')

Recommed News:
--------------------------------------------------
total_sim_score=0.7085087815153607, doc_sim_score=0.4321163136219994, topology_sim_score=0.98490125
('CHT_20200406_1259', '鍾南山警示：兩類無症狀感染者要特別注意 - 兩岸', 'https://www.chinatimes.com/realtimenews/20200403003026-260409')
--------------------------------------------------
total_sim_score=0.6959818330409381, doc_sim_score=0.40901852190539295, topology_sim_score=0.98294514
('CHT_20200331_657', '檢測增加 隔離不力 防疫慢半拍！3大原因 美新冠病例飆至全球首位 - 國際大事', 'https://www.chinatimes.com/newspapers/20200328000492-260119')
--------------------------------------------------
total_sim_score=0.6929542085270844, doc_sim_score=0.40296911413287373, topology_sim_score=0.9829393
('CHT_20200401_330', '隱形的危險 聚焦無症狀感染者 - 焦點新聞', 'https://www.chinatimes.com/newspapers/20200401000180-260301')
----------------------------

### 統計score區間次數

In [156]:
def Standardization(data):
    mu = np.mean(data, axis=0)
    sigma = np.std(data, axis=0)
    print('mean=%f, sigma=%f' %(mu, sigma))
    return (data - mu) / sigma

def Normalization(data):
    _range = np.max(data) - np.min(data)
    return (data - np.min(data)) / _range

In [90]:
pd.Series([k[1] for k in doc_similarity_scores]).value_counts(bins=[0.1*i for i in range(11)], sort=False)#.plot(kind='bar', ylim=(0,5000))

(-0.001, 0.1]    4292
(0.1, 0.2]        865
(0.2, 0.3]         52
(0.3, 0.4]         11
(0.4, 0.5]          4
(0.5, 0.6]          2
(0.6, 0.7]          1
(0.7, 0.8]          1
(0.8, 0.9]          0
(0.9, 1.0]          0
dtype: int64

In [186]:
### 原始doc_similarity_score分數偏低（左尾），標準化 doc_similarity_score
pd.Series(Standardization([k[1] for k in doc_similarity_scores])).value_counts(bins=10,sort=False)#.plot(kind='bar', ylim=(0,2000))

(-1.057, 0.372]     4005
(0.372, 1.785]      1003
(1.785, 3.198]       154
(3.198, 4.611]        46
(4.611, 6.024]        10
(6.024, 7.437]         3
(7.437, 8.85]          3
(8.85, 10.263]         1
(10.263, 11.676]       1
(11.676, 13.089]       2
dtype: int64

In [89]:
pd.Series([k[1] for k in topology_similarity_scores]).value_counts(bins=[0.1*i for i in range(11)], sort=False)#.plot(kind='bar', ylim=(0,5000))

(-0.001, 0.1]       0
(0.1, 0.2]          1
(0.2, 0.3]          0
(0.3, 0.4]          0
(0.4, 0.5]          1
(0.5, 0.6]          0
(0.6, 0.7]          2
(0.7, 0.8]          3
(0.8, 0.9]         67
(0.9, 1.0]       5153
dtype: int64

In [119]:
### 原始topology_similarity_score分數偏高（右尾），標準化 topology_similarity_score
pd.Series(Standardization([k[1] for k in topology_similarity_scores])).value_counts(bins=[0.1*i for i in range(11)],sort=False)#.plot(kind='bar', ylim=(0,2000))

mean=0.983300, sigma=0.029242


(-0.001, 0.1]     371
(0.1, 0.2]        601
(0.2, 0.3]       1041
(0.3, 0.4]       1963
(0.4, 0.5]        106
(0.5, 0.6]          0
(0.6, 0.7]          0
(0.7, 0.8]          0
(0.8, 0.9]          0
(0.9, 1.0]          0
dtype: int64

In [18]:
def Construct_CS_Matrix(context_or_title, News):
    corpus = []
    for i, k in enumerate(News):
        # 結巴中文斷詞
        segments = jieba.cut(k[context_or_title], cut_all=False)
        # 移除停用詞及跳行符號
        remainderWords = list(filter(lambda a: a not in stopWords and a != '\n', segments))
        corpus.append(' '.join(remainderWords))
        
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    cs = cosine_similarity(X)
#     print(cs.shape)
    
    return cs

In [35]:
def Calculate_News_Document_Similarity(NewsIndex1,NewsIndex2,emb_dict):
    def select_CHT_nodes_title_context(database_name, NewsIndex):
        conn = sqlite3.connect(database_name)
        cur = conn.cursor()
        myresult = cur.execute("SELECT NewsIndex,NewsContext,NewsTitle FROM CHTnews WHERE NewsIndex==?", (NewsIndex,))
        result = []
        for row in myresult:
            result.append(row)
        conn.close()    
        return result
    
    News = []
#     for index in tqdm( sorted(list(emb_dict.keys())) ):
#         News.append(list(select_CHT_nodes_title_context(DB_PATH, index)[0]))
    News.append(list(select_CHT_nodes_title_context(DB_PATH, NewsIndex1)[0]))
    News.append(list(select_CHT_nodes_title_context(DB_PATH, NewsIndex2)[0]))
#     print('End loading NewsTitle and NewsContext from database ---> ', time.strftime("%H:%M:%S", time.localtime()) )
        
    # 算 context_or_title 的詞頻矩陣 ---> 1: context , 2: title
    cs1 = Construct_CS_Matrix(1, News)
#     print('End constructing cosine similarity matrix for NewsTitle ---> ', time.strftime("%H:%M:%S", time.localtime()) )
    cs2 = Construct_CS_Matrix(2, News)
#     print('End constructing cosine similarity matrix for NewsContext ---> ', time.strftime("%H:%M:%S", time.localtime()) )
    
    score_context = cs1[0][1]
    score_title = cs2[0][1]
    final_doc_sim_score = max(score_context, score_title)
    
#     doc_similarity_scores = []
#     for i in range(len(cs1)):
#         for j in range(len(cs1)):
#             if j>i:
#                 score_context = cs1[i][j]
#                 score_title = cs2[i][j]
#                 final_doc_sim_score = max(score_context, score_title)
#                 doc_similarity_scores.append([News[i][0], News[j][0], final_doc_sim_score])
    
    return [NewsIndex2, final_doc_sim_score]

In [36]:
def Similar_Document_Content_News(NewsIndex1, emb_dict):
    from tqdm import tqdm
    doc_similarity_scores = []
    for NewsIndex2 in tqdm(emb_dict):
        if NewsIndex2!=NewsIndex1:
            doc_similarity_scores.append(Calculate_News_Document_Similarity(NewsIndex1, NewsIndex2, emb_dict))
            
    return doc_similarity_scores

In [37]:
doc_similarity_scores = Similar_Document_Content_News(NewsID,emb_dict)

100%|██████████| 5229/5229 [04:11<00:00, 20.83it/s]
